In [21]:
import pandas as pd

# my own modules for building risk models, 
# see description here https://github.com/messiah1349/woe_transformer/tree/main
from woe_transformer.utils.model_functions import ModelPipeline

pd.set_option('display.max_columns', None)


In [12]:
df = pd.read_excel('data/DS_test_data.xlsx', index_col='id')
df[:3]

,age,age_rounded,sex,live_place_change,foreign_pass,marriage,children,dependents,education,languages,vehicle,occupational_area,work_by_edu,gov_work,position,family_income,family_member_income,personal_income,credit_sum,branch,city,requested_sum,owner_fact,spouse_income,closed_credits,first_credit,Default60
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,26.936986,27,1,0,0,2,0,0,2,4,3,35,2,3,5,20000,20000.0,20000,60000.0,27,10,60000,5,0,0,0,0
3,31.742466,32,2,0,0,1,2,2,2,4,3,11,3,3,6,22000,5500.0,10000,50000.0,29,12,50000,4,12000,0,1,0
4,23.172603,23,1,0,0,1,1,1,2,3,3,32,1,1,3,15000,5000.0,15000,30000.0,39,1,30000,3,0,1,0,1


In [13]:
df.columns

Index(['age', 'age_rounded', 'sex', 'live_place_change', 'foreign_pass',
       'marriage', 'children', 'dependents', 'education', 'languages',
       'vehicle', 'occupational_area', 'work_by_edu', 'gov_work', 'position',
       'family_income', 'family_member_income', 'personal_income',
       'credit_sum', 'branch', 'city', 'requested_sum', 'owner_fact',
       'spouse_income', 'closed_credits', 'first_credit', 'Default60'],
      dtype='object')

In [29]:
categorical_columns = ['sex', 'work_by_edu', 'gov_work', 'position', 'branch', 'city', 'owner_fact']
target = 'Default60'
numeric_columns = list(set(df.columns) - set(categorical_columns) - set([target]))
feature_names = numeric_columns + categorical_columns

df[categorical_columns] = df[categorical_columns].astype(object)

In [27]:
df[categorical_columns] = df[categorical_columns].astype(object)

In [28]:
ModelPipeline()

id
1       1
3       2
4       1
5       1
6       2
       ..
6042    2
6043    2
6044    2
6045    2
6046    2
Name: sex, Length: 6010, dtype: object